### Variables

In [ ]:
from datasource import DataSource
### Give DataSource a FolderPath, mass, the Complexes Molar Mass and the amount of conc TFA used in 50 mL DCM
### Use 3-5 mg Isoporphyrin in 50 mL DCM and take 10 mL of it. Dilute to 50 mL and do the UV/Vis Titration

#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK038\Titra', 3.5, 899.75245, .1) # OMePy
#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK040\Titra', 4.0, 1082.96245, .2) # OMeDipy
#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK084\Titra', 4.0, 803.73645, .1) # 2ThPy
#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK086\Titra', 4.0, 851.75545, .6) # TolPy
#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK103', 4.0, 779.64845, 2) # PhPy
#src = DataSource(r'C:\Users\jenso\PowerFolders\Forschung\UVVis\JK175\\', 3.5, 851.5321, 1) # 4FPy
#src =  DataSource(r'C:\Users\jenso\PowerFolders\Forschung\practical courses\B Praktikum Chris\UVVis\CK06\3mL TFA Lösung', 4.4,1119.12745,3.0) # A2B3Dipy: OMe/Mes
src =  DataSource(r'C:\Users\jenso\PowerFolders\Forschung\practical courses\B Praktikum Chris\UVVis\CK15\20mL', 4.4, 1056.36192, 20.0) # A2B3Dipy: Ph/Mes
usePH = False
df = src.loadData()

### Settings

In [ ]:
from settings import applySettings
applySettings()

### Plot

In [ ]:
corr = src.calculateCorrection(df)
src.max = 85000 # uncomment to manually adjust scaling
fig,ax = src.plot(corr)
soret_nm = corr.iloc[src.soret][0]
q_nm = corr.iloc[src.q][0]
ph = src.calculatePH(corr, src.q, 1, usePH) # may change to src.q/src.soret, set offset (to 1)  if wrong point is found
print("Soret Band at " + str(soret_nm) + " nm")
print("Q Band at " + str(q_nm) + " nm")
print("pH value is " + str(ph))


### pH Ergebnisse:

In [ ]:
###pH/-log[TFA] /c [mol/L]
OMePy = 0.17 / 2.25 / 0.006
OMeDipy = 0.11 / 1.72 / 0.020
ThPy = 0.14 / 2.05 / 0.009
TolPy = 0.08 / 1.40 / 0.040
PhPy = 0.06 / 1.03 / 0.093
FPy = 0.06 / 1.09 / 0.081
A2B3Dipy_OMe_Mes = 0.04 / 0.54 / 0.288
A2B3Dipy_Ph_Mes = 0.02 / -0.06 / 1.15